## Аналіз A/B-тестів

Ви - аналітик даних в ІТ-компанії і до вас надійшла задача проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це - гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку.

У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми хочемо зрозуміти, як це вплинуло на утримання (retention) гравців. Тобто хочемо зрозуміти, чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Колонки в даних наступні:

- `userid` - унікальний номер, який ідентифікує кожного гравця.
- `version` - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- `sum_gamerounds` - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- `retention_1` - чи через 1 день після встановлення гравець повернувся і почав грати?
- `retention_7` - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

In [5]:
import pandas as pd

df = pd.read_csv(r"D:\моё обучение аналитика\аналітика даних\М15\statistical_hypothesis\cookie_cats.csv")

1. Для початку, уявімо, що ми тільки плануємо проведення зазначеного А/B-тесту і хочемо зрозуміти, дані про скількох користувачів нам треба зібрати, аби досягнути відчутного ефекту. Відчутним ефектом ми вважатимемо збільшення утримання на 1% після внесення зміни. Обчисліть, скільки користувачів сумарно нам треба аби досягнути такого ефекту, якщо продакт менеджер нам повідомив, що базове утримання є 19%.

In [6]:
import statsmodels.stats.api as sms

def sample_size_proportions(p1, p2, alpha=0.05, power=0.8):
    """
    Обчислює необхідний розмір вибірки для A/B тесту з бінарною метрикою.
    
    p1 - базова конверсія (наприклад 0.19)
    p2 - очікувана конверсія (наприклад 0.20)
    alpha - рівень значущості (за замовчуванням 0.05)
    power - потужність тесту (за замовчуванням 0.8)
    """
    effect_size = sms.proportion_effectsize(p1, p2)  # ефект Коена h
    required_n = sms.NormalIndPower().solve_power(
        effect_size=effect_size,
        alpha=alpha,
        power=power,
        alternative='two-sided'
    )
    return int(required_n)

# приклад для retention 19% -> 20%
p1 = 0.19
p2 = 0.20
alpha = 0.05
power = 0.8

n_per_group = sample_size_proportions(p1, p2, alpha, power)
print(f"Необхідний розмір вибірки: {n_per_group} на групу (~{n_per_group*2} сумарно)")

Необхідний розмір вибірки: 24637 на групу (~49274 сумарно)


2. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

In [7]:
# середнє retention_7 по версіях
retention_7_means = df.groupby("version")["retention_7"].mean()
print(retention_7_means)

version
gate_30    0.190201
gate_40    0.182000
Name: retention_7, dtype: float64


H₀ (нульова): Рівень утримання на 7-й день однаковий у групах gate_30 і gate_40.
H₁ (альтернативна): Є відмінність. Якщо беремо направлену гіпотезу → gate_30 має вище утримання, ніж gate_40.

3. Перевірте з допомогою пасуючого варіанту z-тесту, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для варіантів до переміщення воріт і після. Виведіть результат у форматі:

    ```
    z statistic: ...
    p-value: ...
    Довірчий інтервал 95% для групи control: [..., ...]
    Довірчий інтервал 95% для групи treatment: [..., ...]
    ```

    де замість `...` - обчислені значення.
    
    В якості висновку дайте відповідь на два питання:  

      1. Чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
      2. Чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  


In [12]:
import statsmodels.api as sm
import numpy as np

# кількість успіхів (гравців, що повернулись)
success_control = df[df.version=="gate_30"]["retention_7"].sum()
success_treatment = df[df.version=="gate_40"]["retention_7"].sum()

# розмір груп
n_control = df[df.version=="gate_30"].shape[0]
n_treatment = df[df.version=="gate_40"].shape[0]

# Z-тест
z_stat, p_val = sm.stats.proportions_ztest([success_control, success_treatment],
                                           [n_control, n_treatment])

confint_control = sm.stats.proportion_confint(success_control, n_control, alpha=0.05, method='normal')
confint_treatment = sm.stats.proportion_confint(success_treatment, n_treatment, alpha=0.05, method='normal')

print(f"z statistic: {z_stat:.3f}")
print(f"p-value: {p_val:.3f}")
print(f"Довірчий інтервал 95% для групи control: [{confint_control[0]:.3f}, {confint_control[1]:.3f}]")
print(f"Довірчий інтервал 95% для групи treatment: [{confint_treatment[0]:.3f}, {confint_treatment[1]:.3f}]")


z statistic: 3.164
p-value: 0.002
Довірчий інтервал 95% для групи control: [0.187, 0.194]
Довірчий інтервал 95% для групи treatment: [0.178, 0.186]


Статистична значущість

p-value = 0.002 < 0.05 → відхиляємо H₀.

Є значуща різниця між gate_30 і gate_40.

Довірчі інтервали

control (gate_30): [18.7%, 19.4%]

treatment (gate_40): [17.8%, 18.6%]

Інтервали не перетинаються, отже різниця стабільна, не випадкова.

Висновок

Версія з gate_30 дає краще утримання на 7-й день.

Перенесення gate_40  погіршило метрику retention.

4. Виконайте тест Хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та утриманням гравця на 7ий день після реєстрації.

    - Напишіть, як для цього тесту будуть сформульовані гіпотези.
    - Проведіть обчислення, виведіть p-значення і напишіть висновок за результатами тесту.


In [13]:
import pandas as pd
from scipy.stats import chi2_contingency

# створюємо таблицю спряженості (contingency table)
contingency = pd.crosstab(df["version"], df["retention_7"])
print(contingency)

# χ²-тест
chi2, p_val, dof, expected = chi2_contingency(contingency)

print(f"Chi2 statistic: {chi2:.3f}")
print(f"p-value: {p_val:.3f}")
print(f"Degrees of freedom: {dof}")


retention_7  False  True 
version                  
gate_30      36198   8502
gate_40      37210   8279
Chi2 statistic: 9.959
p-value: 0.002
Degrees of freedom: 1


Гіпотези

H₀: Версія гри (gate_30 vs gate_40) не впливає на утримання гравців на 7-й день → змінні незалежні.

H₁: Версія гри впливає на утримання гравців на 7-й день → змінні залежні.

Оскільки p-value = 0.002 < 0.05, ми відхиляємо H₀.
Це означає, що існує статистично значуща залежність між версією гри та ймовірністю утримання гравця на 7-й день.
